<a href="https://colab.research.google.com/github/JITHIN-ANTONY-JOSEPH/ERP_11358080/blob/main/8_Experiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please note , this notebook unlike others has been changed to run for whole datasets if suitable computational power is available.

### Input : Recipe Ingredients
### Model : BERT(Text) + GCN(Graph) + Neural Networks(Model)

### Mounting to connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Install required libraries

In [ ]:
!pip install torch_geometric

### Loading the required datasets

In [ ]:
import pandas as pd
import json

# Load Recipe1M data
with open('/content/drive/My Drive/ERP/modified_Processed_Layer1.json', 'r') as file:
    recipe1m_data = [json.loads(line) for line in file]

recipe1m_df = pd.DataFrame(recipe1m_data)

recipe1m_df = recipe1m_df
recipe1m_df['ingredients'] = recipe1m_df['processed_ingredients'].apply(lambda x: ' '.join(x))


In [ ]:
import networkx as nx

# Load the knowledge graph from the GraphML file
G = nx.read_graphml('/content/drive/My Drive/ERP/knowledge_graph.graphml')

# Extract subgraph with only ingredient nodes
ingredient_nodes = [n for n, attr in G.nodes(data=True) if attr['node_type'] == 'ingredient']
G_ingredients = G.subgraph(ingredient_nodes).copy()

### Initialising BERT

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Generate embeddings for each recipe's ingredients
ingredient_embeddings = {}
for index, row in recipe1m_df.iterrows():
    ingredients_text = row['ingredients']
    inputs = tokenizer(ingredients_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = bert_model(**inputs)
    ingredient_embeddings[row['id']] = outputs.last_hidden_state[:, 0, :].squeeze().detach().numpy()

### Initialising the GCN Model

In [ ]:
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# Convert NetworkX graph to PyTorch Geometric data object
data = from_networkx(G_ingredients)

# Initialize node features randomly
num_features = 64  # Adjust as needed
data.x = torch.randn((data.num_nodes, num_features), dtype=torch.float)

# Move data to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(42)
        self.conv1 = GCNConv(data.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=64)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Train the GCN model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.x)
    loss.backward()
    optimizer.step()
    return loss

for epoch in range(200):
    loss = train()
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Generate node embeddings using the trained GCN model
model.eval()
with torch.no_grad():
    gcn_embeddings = model(data).cpu().numpy()

# Create a mapping from node name to GCN embeddings
node_to_embedding = {name: gcn_embeddings[i] for i, name in enumerate(G_ingredients.nodes())}


### Generate combined embeddings

In [ ]:
import numpy as np

# Combine text and graph embeddings for each ingredient
combined_embeddings = {}
for ingredient in node_to_embedding.keys():
    # Get graph embedding
    graph_embedding = node_to_embedding.get(ingredient, np.zeros(64))  # Assuming GCN embedding size is 64

    # Get text embeddings of all recipes containing the ingredient
    text_embedding_list = []
    for index, row in recipe1m_df.iterrows():
        if ingredient in row['processed_ingredients']:
            text_embedding_list.append(ingredient_embeddings[row['id']])

    # Average text embeddings
    if text_embedding_list:
        text_embedding_avg = np.mean(text_embedding_list, axis=0)
    else:
        text_embedding_avg = np.zeros(768)  # Assuming BERT embedding size is 768

    # Combine graph and text embeddings
    combined_embeddings[ingredient] = np.concatenate((graph_embedding, text_embedding_avg))

### Final Neural Network

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class KeplerModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(KeplerModel, self).__init__()
        self.fc1 = nn.Linear(input_dim * 2, hidden_dim)  # input_dim * 2 because of concatenation
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)

    def forward(self, x1, x2):
        x = torch.cat((x1, x2), dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

input_dim = 832  # 768 (BERT) + 64 (GCN)
hidden_dim = 128
model = KeplerModel(input_dim, hidden_dim)
model = model.to(device)

# Load the substitution pairs
substitution_pairs_df = pd.read_csv('/content/drive/My Drive/ERP/Recipe1MSubs_full.csv')

# Prepare training data
train_data = []
for _, row in substitution_pairs_df.iterrows():
    ingredient1 = row['ingredient1']
    ingredient2 = row['ingredient2']
    if ingredient1 in combined_embeddings and ingredient2 in combined_embeddings:
        embedding1 = combined_embeddings[ingredient1]
        embedding2 = combined_embeddings[ingredient2]
        train_data.append((embedding1, embedding2, 1))

# Add negative samples
import random
all_ingredients = list(combined_embeddings.keys())
for ingredient1 in combined_embeddings.keys():
    for _ in range(5):  # Generate 5 negative samples for each positive sample
        ingredient2 = random.choice(all_ingredients)
        if ingredient1 != ingredient2:
            embedding1 = combined_embeddings[ingredient1]
            embedding2 = combined_embeddings[ingredient2]
            train_data.append((embedding1, embedding2, 0))

# Convert to tensors
X1 = torch.tensor(np.array([t[0] for t in train_data]), dtype=torch.float).to(device)
X2 = torch.tensor(np.array([t[1] for t in train_data]), dtype=torch.float).to(device)
y = torch.tensor([t[2] for t in train_data], dtype=torch.float).to(device).unsqueeze(1)

# Train the model
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

def train_kepler():
    model.train()
    optimizer.zero_grad()
    outputs = model(X1, X2)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Training loop
for epoch in range(100):
    loss = train_kepler()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')

### Evaluation

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Function to generate predictions
def generate_kepler_predictions(validation_pairs, model, embeddings):
    model.eval()
    predictions = []
    with torch.no_grad():
        for _, row in validation_pairs.iterrows():
            ingredient1 = row['ingredient1']
            if ingredient1 in embeddings:
                candidates = sorted(
                    embeddings.keys(),
                    key=lambda ing: model(
                        torch.tensor(embeddings[ingredient1], dtype=torch.float).unsqueeze(0).to(device),
                        torch.tensor(embeddings[ing], dtype=torch.float).unsqueeze(0).to(device)
                    ).item(),
                    reverse=True
                )[:10]
                predictions.append(candidates)
            else:
                predictions.append([])
    return predictions

val_predictions = generate_kepler_predictions(substitution_pairs_df, model, combined_embeddings)

# Function to calculate MRR, Hit@1, Hit@3, Hit@10
def calculate_metrics(predictions, ground_truths):
    mrr = 0.0
    hit_1 = 0.0
    hit_3 = 0.0
    hit_10 = 0.0
    for pred, gt in zip(predictions, ground_truths):
        for i, candidate in enumerate(pred):
            if gt == candidate:
                rank = i + 1
                mrr += 1.0 / rank
                if rank == 1:
                    hit_1 += 1.0
                if rank <= 3:
                    hit_3 += 1.0
                if rank <= 10:
                    hit_10 += 1.0
                break
    mrr /= len(ground_truths)
    hit_1 /= len(ground_truths)
    hit_3 /= len(ground_truths)
    hit_10 /= len(ground_truths)
    return mrr, hit_1, hit_3, hit_10

# Extract ground truths from the validation set
val_ground_truths = substitution_pairs_df['ingredient2'].tolist()

# Calculate metrics for the Kepler model
mrr, hit_1, hit_3, hit_10 = calculate_metrics(val_predictions, val_ground_truths)

print(f"Kepler Model: MRR: {mrr:.4f}, Hit@1: {hit_1:.4f}, Hit@3: {hit_3:.4f}, Hit@10: {hit_10:.4f}")


In [ ]:
from jellyfish import jaro_winkler_similarity

# Function to calculate MRR, Hit@1, Hit@3, and Hit@10 with Jaro-Winkler similarity threshold
def calculate_metrics(predictions, ground_truths, embeddings, threshold=0.8):
    mrr = 0.0
    hit_1 = 0.0
    hit_3 = 0.0
    hit_10 = 0.0

    for pred, gt in zip(predictions, ground_truths):
        for i, candidate in enumerate(pred):
            if gt in embeddings and candidate in embeddings:
                sim = jaro_winkler_similarity(gt, candidate)
                if sim >= threshold:
                    rank = i + 1
                    mrr += 1.0 / rank
                    if rank == 1:
                        hit_1 += 1.0
                    if rank <= 3:
                        hit_3 += 1.0
                    if rank <= 10:
                        hit_10 += 1.0
                    break

    mrr /= len(ground_truths)
    hit_1 /= len(ground_truths)
    hit_3 /= len(ground_truths)
    hit_10 /= len(ground_truths)
    return mrr, hit_1, hit_3, hit_10

# Extract ground truths from the validation set
val_ground_truths = substitution_pairs_df['ingredient2'].tolist()

# Calculate metrics for the Kepler model with Jaro-Winkler similarity threshold
mrr, hit_1, hit_3, hit_10 = calculate_metrics(val_predictions, val_ground_truths, combined_embeddings)

print(f"Kepler Model: MRR: {mrr:.4f}, Hit@1: {hit_1:.4f}, Hit@3: {hit_3:.4f}, Hit@10: {hit_10:.4f}")